In [1]:
# Load Python Packages
# 1. Basic Data Processing Packages
import pandas as pd
import numpy as np
import math
import pickle

# 2. Machine Learning Packages (Modeling, Clustering, Errors)
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
from scipy.cluster.hierarchy import fcluster, dendrogram, linkage
from scipy.spatial.distance import squareform

# 3. Deep Learning Package
import tensorflow as tf

# 4. Other Packages
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

### 01-1. Experiments Setting

In [2]:
# Global Variables
INSTANCE_LIST = ['g4dn.xlarge', 'g5.xlarge']
ANCHOR_INSTANCE = INSTANCE_LIST[0] # Anchor Instance Setting
PRED_INSTANCES = [x for x in INSTANCE_LIST if x != ANCHOR_INSTANCE]
ANCHOR_NAME = ANCHOR_INSTANCE[:2]

CLUSTER_METHOD = 'average'
CLUSTER_THRESHOLD = 20

In [3]:
# Load and check anchor data
pd.set_option('display.max_columns', 137)
pd.set_option('display.max_rows', 13)

anchor_data = pickle.load(open(f"../../dataset/anchor-data/anchor_{ANCHOR_NAME}.pickle", 'rb'))
anchor_data

,Host_OneHot,Device_Log1p,Host_Floor,Device_Sub,Device_Softmax,Host_SelectV2,Host_TensorSliceDataset,Host_Shape,Host_Dataset,Host_Identity,Device_Equal,Host_Cast,Device_ConcatV2,Host_ConcatV2,Device__Send,Device_Split,Host_WriteSummary,Device_DivNoNan,Host_TensorDataset,Host_RegexSplitWithOffsets,Host_Sum,Device_Einsum,Host_AddV2,Host__HostSend,Host_CaseFoldUTF8,Device_Sum,Host_NormalizeUTF8,Host__Send,Host_StopGradient,Device_GreaterEqual,Device__Recv,Device_StridedSlice,Host_GatherV2,Device_LogicalAnd,Host_BroadcastTo,Device_Reciprocal,Device_Pow,Device_ResourceGather,Host_RaggedTensorToTensor,Device_AddV2,Host_Maximum,Device_Unique,Device_Square,Host_LessEqual,Device_TanhGrad,Host_Range,Host_RaggedTensorFromVariant,Device_ReverseV2,Host_Slice,Device_Pack,Device_Sigmoid,Host__Recv,Host_BroadcastArgs,Host_AssignVariableOp,Device_L2Loss,Device_SigmoidGrad,Device_OneHot,Device_Slice,Host_ReadVariableOp,Device_Tanh,Host_TensorListFromTensor,Device_IDLE,Host_RaggedTensorToVariant,Device_Exp,Device_ArgMax,Device_MatMul,Host_Sub,Host__HostRecv,Device_Cast,Device_ResourceScatterAdd,Device_TensorListStack,Device_StridedSliceGrad,Host_LogicalOr,Device_RealDiv,Device_Mean,Device_ResourceApplyAdadelta,Device_Fill,Host_Min,Host_Equal,Host_StridedSlice,Device_Greater,Device_Neg,Device_UnsortedSegmentSum,Device_DynamicStitch,Device_BiasAddGrad,Device_Mul,Host_RealDiv,Device_SquaredDifference,Device__HostRecv,Device_Select,Host_LookupTableFindV2,Host_Mul,Host_Minimum,Device_AssignAddVariableOp,Host_GreaterEqual,Device_ZerosLike,Host_StaticRegexReplace,Host_ConcatenateDataset,Host_ExpandDims,Host_FlushSummaryWriter,Host_Transpose,Host_Reshape,Device_BiasAdd,Host_TensorListStack,Device_RsqrtGrad,Host_LogicalAnd,Device_ResourceSparseApplyAdadelta,Device_CudnnRNN,Device_RandomUniform,Device_AddN,Device_CudnnRNNBackprop,Host_IDLE,Host_IteratorGetNext,Device_Sqrt,Device_SoftmaxCrossEntropyWithLogits,Host_NoOp,Device_ResourceApplyAdam,Host_Max,Host_ZerosLike,Device_ReadVariableOp,Host_Pack,Device_TensorListFromTensor,Device_Transpose,Device_Tile,Device_BroadcastTo,Device_AssignSubVariableOp,Host_RaggedGather,Host_WordpieceTokenizeWithOffsets,exp_name,instance_name,dataset,model,optimizer,batchsize,epoch_latency,batch_latency,g5.xlarge
0,0.000,6.528,0.000,7.199,17.696,0.000,0.000,0.000,464.682,3.637,5.216,0.000,212.665,0.000,20.640,63.325,25.358,17.087,0.000,0.000,0.000,0.000,0.000,0.000,0.000,10.048,0.000,354.215,0.000,12.031,39.647,0.000,127.738,2.400,0.000,5.344,0.000,40.639,0.000,11.712,0.000,347.576,0.000,0.000,0.000,0.000,0.000,153.788,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,10.367,9.008,0.000,0.00,34864.902,0.000,4.320,24.096,42.496,0.000,0.000,37.792,0.000,0.000,92.990,0.000,4.095,4.096,178.458,6.751,0.000,0.000,0.000,0.000,14.240,118.301,8.672,7.200,52.607,0.000,0.000,28.256,23.967,0.000,0.000,0.000,29.822,0.000,104.860,0.00,0.000,0.000,44.060,0.000,0.000,6.240,0.000,0.000,49.673,33.375,7366.530,8.064,339.544,10673.535,1.430806e+05,30.981,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,873.586,5.312,6.528,0.000,0.000,0.000,exp10,g4dn.xlarge,reuters,bilstm,Adadelta,128,1.238734,0.015138,10034.084320
1,2214.833,0.000,41.909,0.000,1381.661,545.914,13.261,54.743,15101.474,196.484,367.448,302.087,780.691,73.973,201.753,0.000,177.774,1478.215,4.674,0.000,934.688,0.000,102.398,963.751,0.000,715.254,0.000,6978.883,12.534,0.000,31804.571,0.000,5206.673,54.973,19.512,0.000,0.000,1445.446,0.000,38743.466,86.377,35070.890,0.000,16.888,40395.698,32.844,0.000,0.000,20.928,0.000,0.000,0.000,14.990,35.643,0.000,0.000,0.000,815.634,771.503,33633.309,0.00,5021289.777,0.000,0.000,1397.535,593050.487,164.355,0.000,1295.711,0.000,8050.206,2707.494,16.802,0.000,0.000,14551.009,3203.678,65.117,83.677,192.343,0.000,0.000,7497.045,0.000,39273.955,828.308,97.492,0.000,43470.517,0.000,0.000,35.425,32.334,2362.352,21.823,55067.266,0.00,17.009,12.542,160.484,48.714,44.720,29612.080,0.000,0.000,431.744,1273.820,0.000,0.000,189334.407,0.000,6.682301e

In [4]:
# Get variables from anchor dataset for clustering and modeling

columns = list(anchor_data.columns)
host_cols = [x for x in columns if x.startswith('Host_')]
device_cols = [x for x in columns if x.startswith('Device_')]
latency_cols = ['epoch_latency', 'batch_latency']
workload_cols = ['instance_name', 'model', 'dataset', 'optimizer', 'batchsize', 'exp_name']

model_list = ['bilstm', 'lstm', 'rnn']
dataset_list = sorted(list(anchor_data['dataset'].value_counts().index))
batchsize_list = sorted(list(anchor_data['batchsize'].value_counts().index))
exp_list = sorted(list(anchor_data['exp_name'].value_counts().index))

### 01-2. Feature Engineering with NLP Clustering

In [5]:
# Function-1: levenshtein
# calculate levenshtein distance of two string (str_x, str_y)

def levenshtein(str_x, str_y):
    size_x = len(str_x) + 1
    size_y = len(str_y) + 1
    matrix = np.zeros((size_x, size_y))
    
    for x in range(size_x):
        matrix[x, 0] = x
    for y in range(size_y):
        matrix[0, y] = y
    for x in range(1, size_x):
        for y in range(1, size_y):
            if str_x[x-1] == str_y[y-1]:
                matrix[x, y] = min(matrix[x-1, y]+1, matrix[x-1, y-1], matrix[x, y-1]+1)
            else:
                matrix[x, y] = min(matrix[x-1, y]+1, matrix[x-1, y-1]+1, matrix[x, y-1]+1)
    return matrix[size_x-1, size_y-1]

In [6]:
# Calculate levenshtein distance matrix for every device operation pairs

feature_names = [x[7:] for x in device_cols] # remove "Device_" character
dist_matrix = pd.DataFrame(0, index=feature_names, columns=feature_names)
for x in feature_names:
    for y in feature_names:
        dist_matrix[x][y] = levenshtein(x, y)

In [7]:
# Apply hierarchical clustering to distance matrix

cluster = fcluster(linkage(squareform(dist_matrix), CLUSTER_METHOD), 10, criterion='distance')
cluster_feature = {i: [] for i in range(len(pd.DataFrame(cluster).value_counts()))}
for index, value in enumerate(feature_names):
    cluster_feature[cluster[index]-1].append(value)

In [8]:
anchor_data = pickle.load(open(f"../../dataset/anchor-data/anchor_{ANCHOR_NAME}.pickle", 'rb'))

for key, value in cluster_feature.items():
    value = ["Device_" + x for x in value]
    
    # 피처 클러스터링 오류 수정
    if len(value) == 1:
        continue
    else:
        anchor_data["&".join(value)] = 0
        for feature in value:
            anchor_data["&".join(value)] += anchor_data[feature]
        anchor_data.drop(value, axis=1, inplace=True)
anchor_data

,Host_OneHot,Host_Floor,Host_SelectV2,Host_TensorSliceDataset,Host_Shape,Host_Dataset,Host_Identity,Host_Cast,Host_ConcatV2,Host_WriteSummary,Host_TensorDataset,Host_RegexSplitWithOffsets,Host_Sum,Host_AddV2,Host__HostSend,Host_CaseFoldUTF8,Host_NormalizeUTF8,Host__Send,Host_StopGradient,Host_GatherV2,Host_BroadcastTo,Host_RaggedTensorToTensor,Host_Maximum,Host_LessEqual,Host_Range,Host_RaggedTensorFromVariant,Host_Slice,Host__Recv,Host_BroadcastArgs,Host_AssignVariableOp,Host_ReadVariableOp,Host_TensorListFromTensor,Host_RaggedTensorToVariant,Host_Sub,Host__HostRecv,Host_LogicalOr,Host_Min,Host_Equal,Host_StridedSlice,Device_UnsortedSegmentSum,Device_DynamicStitch,Host_RealDiv,Device_SquaredDifference,Host_LookupTableFindV2,Host_Mul,Host_Minimum,Host_GreaterEqual,Host_StaticRegexReplace,Host_ConcatenateDataset,Host_ExpandDims,Host_FlushSummaryWriter,Host_Transpose,Host_Reshape,Host_TensorListStack,Host_LogicalAnd,Device_RandomUniform,Device_CudnnRNNBackprop,Host_IDLE,Host_IteratorGetNext,Device_SoftmaxCrossEntropyWithLogits,Host_NoOp,Host_Max,Host_ZerosLike,Host_Pack,Host_RaggedGather,Host_WordpieceTokenizeWithOffsets,exp_name,instance_name,dataset,model,optimizer,batchsize,epoch_latency,batch_latency,g5.xlarge,Device_ResourceGather&Device_ResourceScatterAdd,Device_ResourceApplyAdadelta&Device_ResourceSparseApplyAdadelta&Device_ResourceApplyAdam,Device_TensorListStack&Device_TensorListFromTensor,Device_AssignAddVariableOp&Device_ReadVariableOp&Device_AssignSubVariableOp,Device_StridedSlice&Device_StridedSliceGrad,Device_Log1p&Device_Sub&Device_Softmax&Device_Equal&Device_ConcatV2&Device__Send&Device_Split&Device_DivNoNan&Device_Einsum&Device_Sum&Device_GreaterEqual&Device__Recv&Device_LogicalAnd&Device_Reciprocal&Device_Pow&Device_AddV2&Device_Unique&Device_Square&Device_TanhGrad&Device_ReverseV2&Device_Pack&Device_Sigmoid&Device_L2Loss&Device_SigmoidGrad&Device_OneHot&Device_Slice&Device_Tanh&Device_IDLE&Device_Exp&Device_ArgMax&Device_MatMul&Device_Cast&Device_RealDiv&Device_Mean&Device_Fill&Device_Greater&Device_Neg&Device_BiasAddGrad&Device_Mul&Device__HostRecv&Device_Select&Device_ZerosLike&Device_BiasAdd&Device_RsqrtGrad&Device_CudnnRNN&Device_AddN&Device_Sqrt&Device_Transpose&Device_Tile&Device_BroadcastTo
0,0.000,0.000,0.000,0.000,0.000,464.682,3.637,0.000,0.000,25.358,0.000,0.000,0.000,0.000,0.000,0.000,0.000,354.215,0.000,127.738,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,9.008,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,118.301,8.672,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,44.060,0.000,0.000,0.000,49.673,8.064,10673.535,1.430806e+05,30.981,0.000,0.000,0.000,0.000,0.000,0.000,0.000,exp10,g4dn.xlarge,reuters,bilstm,Adadelta,128,1.238734,0.015138,10034.084320,40.639,211.833,0.000,29.822,92.990,44760.687
1,2214.833,41.909,545.914,13.261,54.743,15101.474,196.484,302.087,73.973,177.774,4.674,0.000,934.688,102.398,963.751,0.000,0.000,6978.883,12.534,5206.673,19.512,0.000,86.377,16.888,32.844,0.000,20.928,0.000,14.990,35.643,771.503,0.00,0.000,164.355,0.000,16.802,65.117,83.677,192.343,7497.045,0.000,97.492,0.000,0.000,35.425,32.334,21.823,0.00,17.009,12.542,160.484,48.714,44.720,0.000,431.744,0.000,0.000,6.682301e+07,714.439,5135.306,4.794,43.480,16.849,58.996,0.000,0.000,exp10,g4dn.xlarge,babi,rnn,Adadelta,128,3.618825,0.045379,47852.516174,1445.446,15824.829,18980.021,2365.199,2707.494,6168787.000
2,0.000,0.000,0.000,0.000,0.000,465.425,3.777,0.000,0.000,23.810,0.000,0.000,0.000,0.000,16.434,0.000,0.000,416.029,0.000,160.294,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,9.184,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,241.500,21.856,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.000,43.049,0.000,0.000,0.000,41.010,0.000,0.000,7.432192e+05,21.165,0.000,0.000,0.000,0.000,0.000,0.000,0.000,exp10,g4dn.xlarge,reuters,rnn,Adadelta,128,5.869380,0.079334,83506.345749,48.863,269.369,758.382,60.222,55.743,245988.103
3,0.000,0.000,0.000,0.000,0.000,493.907,4.017,0.000,0.000,23.5

### 01-3. Modeling and Validation

In [9]:
# define deep neural network regression model with custom learning rate, loss, layers

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9)

def build_dnn_model(input_shape):
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(128,activation="relu", input_shape=input_shape))
    model.add(tf.keras.layers.Dense(64,activation="relu"))
    model.add(tf.keras.layers.Dense(32,  activation="relu"))
    model.add(tf.keras.layers.Dense(16,  activation="relu"))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                  loss=['mean_absolute_percentage_error', 'mean_squared_error'],
                  loss_weights=[1., 1.])
    return model

In [10]:
# Function-2: train_test_model
# 1. Get train/test dataset and target instance(pred_instance)
# 2. Split dataset into feature and target (x, y)
# 3. train three single model (linear, random forest, dnn)
# 4. predict with three model, then median ensemble to results
# 5. return prediction results

def train_test_model(train_data, test_data, pred_instance, drop_cols):
    train_x = train_data.drop(drop_cols + PRED_INSTANCES, axis=1)
    train_simple_x = np.array(train_data['batch_latency']).reshape(-1, 1)
    train_y = train_data[pred_instance]
    
    test_x = test_data.drop(drop_cols + PRED_INSTANCES, axis=1)
    test_simple_x = np.array(test_data['batch_latency']).reshape(-1, 1)
    test_y = test_data[[pred_instance]].to_numpy()
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
    model_dnn = build_dnn_model((train_x.shape[1],))
    model_dnn.fit(train_x, train_y, epochs=200,
              callbacks=[callback],
              batch_size=16,
              verbose=0)
    model_rfr = RandomForestRegressor()
    model_rfr.fit(train_x, train_y)
    model_simple = LinearRegression()
    model_simple.fit(train_simple_x, train_y)
    
    dnn_pred_y = model_dnn.predict(test_x)
    dnn_pred_y = dnn_pred_y.reshape(-1, 1)
    rfr_pred_y = model_rfr.predict(test_x)
    rfr_pred_y = rfr_pred_y.reshape(-1, 1)
    simple_pred_y = model_simple.predict(test_simple_x)
    simple_pred_y = simple_pred_y.reshape(-1, 1)
    median_pred_y = np.median(np.stack([
        dnn_pred_y, rfr_pred_y, simple_pred_y
    ]), axis=0)
    
#     print('dnn_pred_y',dnn_pred_y)
#     print('rfr_pred_y', rfr_pred_y)
#     print('simple_pred_y',simple_pred_y)
#     print('median_pred_y', median_pred_y)
    
    return test_y, dnn_pred_y,rfr_pred_y,simple_pred_y,median_pred_y, test_data

In [11]:
# Function-3: model_validation
# 1. run model-validation and save result to dictionary of dictionary
#    {'g3s.xlarge': {'AlexNet': ... , 'VGG16': ...}, 'g4dn.xlarge': {...}}
# 2. split traing and test dataset by model(dnn-architecture) condition
# 3. execute train_test_model function, the save result to dictionary
from sklearn.preprocessing import MinMaxScaler

def model_validation():
    pred_instance_dict = {}
    for pred_instance in PRED_INSTANCES:
        pred_model_dict = {}
        for val_model in model_list:
            
            train_data = anchor_data[(anchor_data['dataset'] == 'reuters')&(anchor_data['model'] != 'smallbert')]            
            test_data = anchor_data[(anchor_data['dataset'] == 'reuters')&(anchor_data['model'] != 'smallbert')]            

            print(f"Validation Model: {val_model}")
            print(f"Train Data Size: {train_data['model']}")
            print(f"Test Data Size: {test_data['model']}")
            print('-------------------------------------')
            test_y, dnn_pred_y, rfr_pred_y, simple_pred_y, median_pred_y, test_data = train_test_model(
                train_data, test_data, pred_instance,
                latency_cols + workload_cols + PRED_INSTANCES)
            
            pred_model_dict[val_model] = (test_y, dnn_pred_y, rfr_pred_y, simple_pred_y, median_pred_y, test_data)
        pred_instance_dict[pred_instance] = pred_model_dict
        return pred_instance_dict

In [12]:
pred_instance_dict = model_validation()

Validation Model: bilstm
Train Data Size: 0    bilstm
2       rnn
4      lstm
Name: model, dtype: object
Test Data Size: 0    bilstm
2       rnn
4      lstm
Name: model, dtype: object
-------------------------------------
Validation Model: lstm
Train Data Size: 0    bilstm
2       rnn
4      lstm
Name: model, dtype: object
Test Data Size: 0    bilstm
2       rnn
4      lstm
Name: model, dtype: object
-------------------------------------
Validation Model: rnn
Train Data Size: 0    bilstm
2       rnn
4      lstm
Name: model, dtype: object
Test Data Size: 0    bilstm
2       rnn
4      lstm
Name: model, dtype: object
-------------------------------------


### 01-4. Preprocessing Validation Results

In [13]:
# Function-4: build_true_pred_df
# convert dictionary to dataframe

def build_true_pred_df(pred_instance_dict, instance, model):
    true_y = pred_instance_dict[instance][model][0]
    dnn_y = pred_instance_dict[instance][model][1]
    test_df = pred_instance_dict[instance][model][5]

    pred_df = test_df[['model', 'dataset', 'batchsize', 'exp_name']]
    pred_df[f"{instance}_true"] = true_y
    pred_df['dnn_y'] = dnn_y
    pred_df['batchsize'] = pd.to_numeric(pred_df['batchsize'])
    pred_df['anchor_mape'] = abs((true_y - dnn_y) / true_y) * 100

    return pred_df

In [14]:
all_result_true_pred_df = []
for test_instance in PRED_INSTANCES:
    single_target_true_pred_df_list = []
    for test_model in model_list:
        true_pred_df = build_true_pred_df(pred_instance_dict, test_instance, test_model)
        single_target_true_pred_df_list.append(true_pred_df)
    single_target_true_pred_df = pd.concat(single_target_true_pred_df_list)
    all_result_true_pred_df.append(single_target_true_pred_df)  
    
# pd.set_option('display.max_rows', None)
print(all_result_true_pred_df[0][:])

    model  dataset  batchsize exp_name  g5.xlarge_true          dnn_y  \
0  bilstm  reuters        128    exp10    10034.084320   15297.963867   
2     rnn  reuters        128    exp10    83506.345749   80945.132812   
4    lstm  reuters        128    exp10   393855.333328  439563.875000   
0  bilstm  reuters        128    exp10    10034.084320   11254.816406   
2     rnn  reuters        128    exp10    83506.345749   62683.386719   
4    lstm  reuters        128    exp10   393855.333328  341492.156250   
0  bilstm  reuters        128    exp10    10034.084320   14108.250000   
2     rnn  reuters        128    exp10    83506.345749   75092.539062   
4    lstm  reuters        128    exp10   393855.333328  405278.812500   

   anchor_mape  
0    52.459989  
2     3.067088  
4    11.605414  
0    12.165854  
2    24.935780  
4    13.295028  
0    40.603263  
2    10.075649  
4     2.900425  
